# Agent Executor

In [1]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langchain4jVersion = "1.9.1";
var langchain4jbeta = "1.9.1-beta17";
var langgraph4jVersion = "1.7-SNAPSHOT";

Remove installed package from Jupiter cache

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

add local maven repo and install dependencies

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.7-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.7-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.7-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:1.9.1
Adding dependency dev.langchain4j:langchain4j-open-ai:1.9.1
Solving dependencies
Resolved artifacts count: 78
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.7-SNAPSHOT/langgraph4j-core-1.7-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/L

**Initialize Logger**

In [4]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    java.util.logging.LogManager.getLogManager().readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AgentExecutor");


Create Tools

In [5]:
import dev.langchain4j.agent.tool.P;
import dev.langchain4j.agent.tool.Tool;

import java.util.Optional;

import static java.lang.String.format;

public class TestTool {
    private String lastResult;

    Optional<String> lastResult() {
        return Optional.ofNullable(lastResult);
    }

    @Tool("tool for test AI agent executor")
    String execTest(@P("test message") String message) {

        lastResult = format( "test tool executed: %s", message);
        return lastResult;
    }
}


In [6]:
import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import org.bsc.langgraph4j.CompileConfig;
import org.bsc.langgraph4j.RunnableConfig;
import org.bsc.langgraph4j.checkpoint.BaseCheckpointSaver;
import org.bsc.langgraph4j.checkpoint.MemorySaver;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.serializer.StateSerializer;

import org.bsc.langgraph4j.agentexecutor.AgentExecutor;

import dev.langchain4j.model.openai.OpenAiChatModel;

import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;



var chatModel = OpenAiChatModel.builder()
    .apiKey( System.getenv("OPENAI_API_KEY") )
    //.modelName( "gpt-3.5-turbo-0125" )
    .modelName( "gpt-4o-mini" )
    .logResponses(true)
    .maxRetries(2)
    .temperature(0.0)
    .maxTokens(2000)
    .build();

var stateGraph = AgentExecutor.builder()
        .chatModel(chatModel)
        .toolsFromObject( new TestTool() )
        .build();




### Test 1 
Update State replacing the 'input'

In [7]:
import dev.langchain4j.data.message.UserMessage;

var saver = new MemorySaver();

CompileConfig compileConfig = CompileConfig.builder()
                .checkpointSaver( saver )
                .build();

var graph = stateGraph.compile( compileConfig );

var config = RunnableConfig.builder()
                .threadId("test1")
                .build();    
                
var iterator = graph.streamSnapshots( Map.of( "messages", UserMessage.from("perform test once")), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



START 
callAgent 
STEP: StateSnapshot{node=__START__, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }
	]
	}, config=RunnableConfig{ threadId=test1, checkPointId=0b799e1c-2563-498c-902e-55ce7a3fce88, nextNode=agent, streamMode=SNAPSHOTS }} 
executeTools 
execute: [execTest] 
STEP: StateSnapshot{node=agent, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }
	AiMessage { text = null, thinking = null, toolExecutionRequests = [ToolExecutionRequest { id = "call_KGhXbzdA3z5NNZU0ux2Mwzwo", name = "execTest", arguments = "{"message":"perform test once"}" }], attributes = {} }
	]
	}, config=RunnableConfig{ threadId=test1, checkPointId=f411fa43-6c35-47ae-946f-6a35a56c1d1a, nextNode=action, streamMode=SNAPSHOTS }} 
callAgent 
STEP: StateSnapshot{node=action, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test o

In [8]:
import org.bsc.langgraph4j.state.RemoveByHash;

var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

for( var snapshot : history ) {
    log.info( "{}", snapshot );
}


// var state2 =  history.get(2);

// var updatedConfig = graph.updateState( state2.config(), 
//     Map.of( "messages", UserMessage.from("perform test twice")), null);

// var iterator = graph.streamSnapshots( null, updatedConfig );  


// for( var step : iterator ) {
//     log.info( "STEP:\n {}", step );
// }


StateSnapshot{node=action, state={
	agent_response=no tool execution request found!
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }
	AiMessage { text = null, thinking = null, toolExecutionRequests = [ToolExecutionRequest { id = "call_KGhXbzdA3z5NNZU0ux2Mwzwo", name = "execTest", arguments = "{"message":"perform test once"}" }], attributes = {} }
	ToolExecutionResultMessage { id = "call_KGhXbzdA3z5NNZU0ux2Mwzwo" toolName = "execTest" text = "test tool executed: perform test once" }
	AiMessage { text = "The test has been executed with the message: "perform test once".", thinking = null, toolExecutionRequests = [], attributes = {} }
	]
	}, config=RunnableConfig{ threadId=test1, checkPointId=8cf9d417-73a2-4ef6-b50b-2bbc0d0adfcb, nextNode=__END__, streamMode=VALUES }} 
StateSnapshot{node=agent, state={
	agent_response=The test has been executed with the message: "perform test once".
	messages=[
	UserMessage { name = null, co

### Test 2
Update State replacing the 'input' using a cloned state

In [9]:

var config = RunnableConfig.builder()
                .threadId("test2")
                .build();    
                
                
var iterator = graph.streamSnapshots( Map.of( "messages", UserMessage.from("perform test once") ), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



START 
callAgent 
STEP: StateSnapshot{node=__START__, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }
	]
	}, config=RunnableConfig{ threadId=test2, checkPointId=285b5856-031f-465d-8136-66dfb902733c, nextNode=agent, streamMode=SNAPSHOTS }} 
executeTools 
execute: [execTest] 
STEP: StateSnapshot{node=agent, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }
	AiMessage { text = null, thinking = null, toolExecutionRequests = [ToolExecutionRequest { id = "call_hq3RECYLLcBGkVxTuRdGPQ1k", name = "execTest", arguments = "{"message":"perform test once"}" }], attributes = {} }
	]
	}, config=RunnableConfig{ threadId=test2, checkPointId=61a1df1a-cca3-4391-83a7-980f6c409c3c, nextNode=action, streamMode=SNAPSHOTS }} 
callAgent 
STEP: StateSnapshot{node=action, state={
	messages=[
	UserMessage { name = null, contents = [TextContent { text = "perform test o

In [10]:
var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

var state2 =  history.get(2);

var updatedState = new HashMap<String,Object>();
updatedState.putAll(state2.state().data());

updatedState.put(  "messages", UserMessage.from("perform test twice")  );

// var updatedConfig = graph.updateState( state2.config(), updatedState );
// log.info( "UPDATED CONFIG: {}", updatedConfig );

// var iterator = graph.streamSnapshots( null, updatedConfig );  

// for( var step : iterator ) {
//     log.info( "STEP:\n {}", step );
// }    


[UserMessage { name = null, contents = [TextContent { text = "perform test once" }], attributes = {} }, AiMessage { text = null, thinking = null, toolExecutionRequests = [ToolExecutionRequest { id = "call_hq3RECYLLcBGkVxTuRdGPQ1k", name = "execTest", arguments = "{"message":"perform test once"}" }], attributes = {} }, ToolExecutionResultMessage { id = "call_hq3RECYLLcBGkVxTuRdGPQ1k" toolName = "execTest" text = "test tool executed: perform test once" }]